In [1]:
# Paso 3 con mapeo de cuantiles.
# Paso 2 sin mapeo de cuantiles.
# Obtiene la variable DNI (Direct Normal Irradiance)
# a partir de GHI usando el modelo DISC de NREL.

import os

import numpy as np
import pandas as pd

import xarray as xr

# Inicializamos el dashboard de cómputo distribuido.
from dask.distributed import Client
c_lat = 1
c_lon = 1
Client( n_workers = 2, threads_per_worker = 5, memory_limit = "3.5GB" )

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 10,Total memory: 6.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52310,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 6.52 GiB
Comm: tcp://127.0.0.1:52318,Total threads: 5
Dashboard: http://127.0.0.1:52319/status,Memory: 3.26 GiB
Nanny: tcp://127.0.0.1:52313,


In [3]:
# Cargamos el archivo.
path_d = "../temp/radiacion/"
path_r = "../temp/TMY_prep/"
#f_d = "WRF_miroc_1985_2014_radiacion.nc"
#f_r = "WRF_miroc_1985_2014_radiacion_days.nc"
f_d = "WRF_miroc_1985_2014_00096_nan.nc"
f_r = "WRF_miroc_1985_2014_00096_nan_days.nc"
vnames = [ "GHI", "DNI", "T_min", "T_max", "T_mean", 
        "Dp_min", "Dp_max", "Dp_mean", "W_max", "W_mean" ]

files = os.listdir(path_d)
files.sort()
if ".DS_Store" in files: files.remove(".DS_Store")
for f in files:
	print( f"{f[-5:-3]}", end = "," )
	with xr.open_dataset( path_d + f,
	chunks = { "lat": c_lat, "lon": c_lon },
	drop_variables = [ "Pressure", "Wind Direction",
	"P_mp", "Relative Humidity" ] ) as ds:

		# Obtenemos la información diaria.
		ds_d = ds[ [vnames[0]] ].resample({"time": "1D"}).sum()
		ds_d[vnames[1]] = ds[ vnames[1]     ].resample({"time": "1D"}).sum()
		ds_d[vnames[2]] = ds[ "Temperature" ].resample({"time": "1D"}).min()
		ds_d[vnames[3]] = ds[ "Temperature" ].resample({"time": "1D"}).max()
		ds_d[vnames[4]] = ds[ "Temperature" ].resample({"time": "1D"}).mean()
		ds_d[vnames[5]] = ds[ "Dew Point"   ].resample({"time": "1D"}).min()
		ds_d[vnames[6]] = ds[ "Dew Point"   ].resample({"time": "1D"}).max()
		ds_d[vnames[7]] = ds[ "Dew Point"   ].resample({"time": "1D"}).mean()
		ds_d[vnames[8]] = ds[ "Wind Speed"  ].resample({"time": "1D"}).max()
		ds_d[vnames[9]] = ds[ "Wind Speed"  ].resample({"time": "1D"}).mean()

		ds_d.to_netcdf(path_r + f, mode = "w")